In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [156]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [157]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

print("--Get data--")
dataframe = pd.read_csv('heart.csv')
data_file_train = pd.read_csv('heart_train.csv')
data_file_test = pd.read_csv('heart_test.csv')
#data_file_train, data_file_test = train_test_split(dataframe, test_size = 0.15)
data_file_train.head()
dataframe.describe()





--Get data--


,row.names,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,231.935065,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,133.938585,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,1.000000,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,116.250000,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,231.500000,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,347.750000,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,463.000000,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


In [158]:
print("--See data types--")
data_file.dtypes

--See data types--


row.names      int64
sbp            int64
tobacco      float64
ldl          float64
adiposity    float64
famhist       object
typea          int64
obesity      float64
alcohol      float64
age            int64
chd            int64
dtype: object

In [159]:
print("--Convert objects to discrete numerical values")
data_file_train['famhist'] = pd.Categorical(data_file_train['famhist'])
data_file_train['famhist'] = data_file_train.famhist.cat.codes
data_file_test['famhist'] = pd.Categorical(data_file_test['famhist'])
data_file_test['famhist'] = data_file_test.famhist.cat.codes
data_file_train.head()

--Convert objects to discrete numerical values


,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,77,134,0.0,5.90,30.84,0,49,29.16,0.00,55,0
1,78,140,0.6,5.56,33.39,1,58,27.19,0.00,55,1
2,79,168,4.5,6.68,28.47,0,43,24.25,24.38,56,1
3,80,108,0.4,5.91,22.92,1,57,25.72,72.00,39,0
4,81,114,3.0,7.04,22.64,1,55,22.59,0.00,45,1


In [160]:
print("loading data into dataset")
target_test = data_file_test.pop('chd')
target_train = data_file_train.pop('chd')

loading data into dataset


In [161]:
dataset_train = tf.data.Dataset.from_tensor_slices((data_file_train.values, target_train.values))
dataset_test = tf.data.Dataset.from_tensor_slices((data_file_test.values, target_test.values))
train = dataset_train.shuffle(len(data_file_train)).batch(1)
test = dataset_test.shuffle(len(data_file_test)).batch(1)
print(train)


<BatchDataset shapes: ((None, 10), (None,)), types: (tf.float64, tf.int64)>


In [162]:

combined_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=2*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

model.compile(optimizer=get_optimizer(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train, epochs=8, verbose=2)
model_loss, model_acc = model.evaluate(test, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")

combined_model.compile(optimizer=get_optimizer(), loss='binary_crossentropy', metrics=['accuracy'])
combined_model.fit(train, epochs=8, verbose=2)
combined_model_loss, model_acc = combined_model.evaluate(test, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/8
386/386 - 1s - loss: 2.0084 - accuracy: 0.5777
Epoch 2/8
386/386 - 1s - loss: 1.1363 - accuracy: 0.6321
Epoch 3/8
386/386 - 1s - loss: 1.1945 - accuracy: 0.5984
Epoch 4/8
386/386 - 1s - loss: 0.9025 - accuracy: 0.6528
Epoch 5/8
386/386 - 1s - loss: 0.8003 - accuracy: 0.6503
Epoch 6/8
386/386 - 1s - loss: 0.8812 - accuracy: 0.6192
Epoch 7/8
386/386 - 1s - loss: 0.7562 - accuracy: 0.6554
Epoch 8/8
386/386 - 1s - loss: 0.7447 - accuracy: 0.6580
76/76 - 0s - loss: 1.1425 - accuracy: 0.6184
Model L